# Introduction 

In this Jupyter notebook, I present my approach to the recent Kaggle competition, [Facebook Recruiting IV: Human or Robot?](https://www.kaggle.com/c/facebook-recruiting-iv-human-or-bot). 

# Problem

For this competition, we will be chasing down robots for an online auction site. Human bidders on the site are becoming increasingly frustrated with their inability to win auctions vs their software-controlled counterparts. Goal of this competition is to identify online auction bids that are placed by bots, helping the site owners to easily flag these users for removal from their site to prevent unfair auction activity.

# Libaries 

For the most part, I used Python standard libraries and the scientific Python libraries available in the Anaconda distribution (pandas, scikit-learn, scipy and numpy). 
For the libaries, I used the standard libaries in Python ( pandas , numpy , scikit-learn ).


In [94]:
import numpy as np
import pandas as  pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from catboost import CatBoostClassifier      
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
import statistics
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score

# Feature Engineering

## Trdr;
Some of the features we came up with was : <br>
Num of bids<br>
statistics of time difference for bids of each bidder<br>
entropy level of each bidder<br>
standard statistics of characteristics of each bidder (IPs, Devices, Countries, URLs)<br>
Number of 'first' and 'last' bids done for an auction for each bidder


In [95]:
import pandas as  pd
bids_df= pd.read_csv('data/bids.csv')
df_train= pd.read_csv('data/train.csv')
test_df= pd.read_csv('data/test.csv')


In [96]:
bids_df.head()

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3


# Converting columns to numeric values & Computing Bids Per Auction

In [97]:
bids_unique= bids_df.groupby('bidder_id').nunique()

In [98]:
bids_unique = bids_unique.drop(["bid_id",'bidder_id'],axis = 1)

In [184]:
join_result= df_train.merge(bids_unique, how= 'left', on = 'bidder_id')
join_result.fillna(0,inplace = True)

In [100]:
bids_df_count= bids_df.groupby("bidder_id")
counts= bids_df_count['url'].count().reset_index().rename(columns = {'url':'num_bids'})

In [101]:
new_train= join_result.merge(counts, how= 'left')

# Calculating time difference for bidders' bids 

In [103]:
bids_df_sorted= bids_df.sort_values(by= ['bidder_id', 'time'], ascending = [True, True])

In [104]:
timediff= bids_df_sorted.groupby('bidder_id')['time'].diff()

In [105]:
bids_df_sorted['timediff']= timediff

In [106]:
bids= pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [107]:
max_time = bids_df_sorted.groupby('bidder_id')['time'].max()
bids['maxtime_num'] = max_time
min_time = bids_df_sorted.groupby('bidder_id')['time'].min()
bids['mintime_num'] = min_time

In [108]:
max_diff = bids_df_sorted.groupby('bidder_id')['timediff'].max()
max_diff = max_diff.fillna(max_diff.mean())
bids['maxdiff_num'] = max_diff
min_diff = bids_df_sorted.groupby('bidder_id')['timediff'].max()
min_diff = min_diff.fillna(min_diff.mean())
bids['mindiff_num'] = min_diff
mean_diff = bids_df_sorted.groupby('bidder_id')['timediff'].mean()
mean_diff = mean_diff.fillna(mean_diff.mean())
bids['meandiff_num'] = mean_diff
median_diff = bids_df_sorted.groupby('bidder_id')['timediff'].median()
median_diff = median_diff.fillna(median_diff.mean())
bids['mediandiff_num'] = median_diff
std_diff = bids_df_sorted.groupby('bidder_id')['timediff'].std()
std_diff = std_diff.fillna(std_diff.mean())
bids['std_diff'] = std_diff

In [109]:
end_auction_time= bids_df.groupby('auction').time.max().reset_index()
end_auction_time= end_auction_time.rename(columns= {'time': 'endtime'})
start_auction_time= bids_df.groupby('auction').time.min().reset_index()
start_auction_time= start_auction_time.rename(columns= {'time': 'starttime'})
start_end_times= pd.merge(start_auction_time, end_auction_time, on= 'auction', how= 'left')

In [110]:
bids_df_sorted= pd.merge(bids_df_sorted, start_end_times[['auction', 'starttime','endtime']], on='auction', how= 'left')

In [111]:
bids_df_sorted['time_until_end']= bids_df_sorted.endtime- bids_df_sorted.time
bids_df_sorted['time_from_start']= bids_df_sorted.time - bids_df_sorted.starttime

In [112]:
median_start= bids_df_sorted.groupby('bidder_id').time_from_start.median()
bids['median_start'] = median_start
median_start= median_start.fillna(median_start.mean())
mean_start= bids_df_sorted.groupby('bidder_id').time_from_start.mean()
bids['mean_start'] = mean_start
mean_start= mean_start.fillna(mean_start.mean())
std_start= bids_df_sorted.groupby('bidder_id').time_from_start.std()
std_start= std_start.fillna(std_start.mean())
bids['std_start'] = std_start

In [113]:
median_end= bids_df_sorted.groupby('bidder_id').time_until_end.median()
bids['median_end'] = median_end
median_end= median_end.fillna(median_end.mean())
mean_end= bids_df_sorted.groupby('bidder_id').time_until_end.mean()
bids['mean_end'] = mean_end
mean_end= mean_end.fillna(mean_end.mean())
std_end= bids_df_sorted.groupby('bidder_id').time_until_end.std()
std_end= std_end.fillna(std_end.mean())
bids['std_end'] = std_end

# Calculating Entropy for each bidder

In [114]:
def log_entropy(x):
    e = np.sum(np.log(np.array(range(1,np.sum(x)))))
    for i in x:
        e -= np.sum(np.log(np.array(range(1,i))))

    return e


In [115]:
dataweusing= bids_df_sorted[['bidder_id', 'auction', 'ip', 'url', 'country', 'device', 'merchandise']]
dataweusing= dataweusing.groupby(['bidder_id', 'auction'])

In [117]:
bids_per_auction_per_ip= bids_df_sorted[['bidder_id', 'auction', 'ip']].groupby(['bidder_id','auction', 'ip']).size()
bids_per_auction_per_ip2= bids_per_auction_per_ip.groupby(['bidder_id', 'auction']).apply(log_entropy).reset_index()
median_entropy=bids_per_auction_per_ip2.groupby('bidder_id').median()
mean_entropy=bids_per_auction_per_ip2.groupby('bidder_id').mean()
std_entropy=bids_per_auction_per_ip2.groupby('bidder_id').std()
bids['mean_entropy']= mean_entropy
bids['median_entropy']= median_entropy
bids['std_entropy']= std_entropy

# Calculating IPs, countries,devices,urls per bidder per auction

In [118]:
ips_per_bidder_per_auction= dataweusing.ip.nunique()

In [119]:
median_ips_per_bidder_per_auction= ips_per_bidder_per_auction.groupby('bidder_id').median()
mean_ips_per_bidder_per_auction=ips_per_bidder_per_auction.groupby('bidder_id').mean()
std_ips_per_bidder_per_auction=ips_per_bidder_per_auction.groupby('bidder_id').std()
bids['median_ips_per_bidder_per_auction']= median_ips_per_bidder_per_auction
bids['mean_ips_per_bidder_per_auction']= mean_ips_per_bidder_per_auction
bids['std_ips_per_bidder_per_auction']= std_ips_per_bidder_per_auction

ip_count= bids_df_sorted.groupby(['bidder_id', 'ip']).size()
ip_entropy= ip_count.groupby('bidder_id').apply(log_entropy)
bids['ip_entropy']= ip_entropy

In [120]:
country_per_bidder_per_auction= dataweusing.country.nunique()

In [121]:
median_country_per_bidder_per_auction= country_per_bidder_per_auction.groupby('bidder_id').median()
mean_country_per_bidder_per_auction=country_per_bidder_per_auction.groupby('bidder_id').mean()
std_country_per_bidder_per_auction=country_per_bidder_per_auction.groupby('bidder_id').std()
bids['median_country_per_bidder_per_auction']= median_country_per_bidder_per_auction
bids['mean_country_per_bidder_per_auction']= mean_country_per_bidder_per_auction
bids['std_country_per_bidder_per_auction']= std_country_per_bidder_per_auction
country_count= bids_df_sorted.groupby(['bidder_id', 'country']).size()
country_entropy= ip_count.groupby('bidder_id').apply(log_entropy)
bids['country_entropy']= country_entropy

In [122]:
device_per_bidder_per_auction= dataweusing.device.nunique()

In [123]:
median_devices_per_bidder_per_auction= device_per_bidder_per_auction.groupby('bidder_id').median()
mean_devices_per_bidder_per_auction=device_per_bidder_per_auction.groupby('bidder_id').mean()
std_devices_per_bidder_per_auction=device_per_bidder_per_auction.groupby('bidder_id').std()
bids['median_devices_per_bidder_per_auction']= median_devices_per_bidder_per_auction
bids['mean_devices_per_bidder_per_auction']= mean_devices_per_bidder_per_auction
bids['std_devices_per_bidder_per_auction']= std_devices_per_bidder_per_auction
device_count= bids_df_sorted.groupby(['bidder_id', 'device']).size()
device_entropy= ip_count.groupby('bidder_id').apply(log_entropy)
bids['device_entropy']= device_entropy

In [124]:
url_per_bidder_per_auction= dataweusing.url.nunique()

In [125]:
median_url_per_bidder_per_auction= url_per_bidder_per_auction.groupby('bidder_id').median()
mean_url_per_bidder_per_auction=url_per_bidder_per_auction.groupby('bidder_id').mean()
std_url_per_bidder_per_auction=url_per_bidder_per_auction.groupby('bidder_id').std()
bids['median_url_per_bidder_per_auction']= median_url_per_bidder_per_auction
bids['mean_url_per_bidder_per_auction']= mean_url_per_bidder_per_auction
bids['std_url_per_bidder_per_auction']= std_url_per_bidder_per_auction
url_count= bids_df_sorted.groupby(['bidder_id', 'url']).size()
url_entropy= ip_count.groupby('bidder_id').apply(log_entropy)
bids['url_entropy']= device_entropy

In [126]:
new_train= new_train.merge(bids, how = 'left', left_on = ['bidder_id'], right_on = ['bidder_id'])
new_train.corr()
new_train['country'].replace({0:1},inplace=True)

In [127]:
new_train['bids_per_auction'] = new_train['num_bids']/new_train['auction']
new_train['bids_per_url'] = new_train['num_bids']/new_train['url']
new_train['bids_per_ip'] = new_train['num_bids']/new_train['ip']
new_train['bids_per_country'] = new_train['num_bids']/new_train['country']
new_train['bids_per_device'] = new_train['num_bids']/new_train['device']


# Calculating country, devices, ips , urls per auction per bidder

In [129]:
diversity_bids = bids_df.sort_values(['bidder_id','country'])

In [130]:
num_countries_per_auction = diversity_bids.groupby(['bidder_id','auction'])['country'].nunique()
num_countries_per_auction = pd.DataFrame(num_countries_per_auction)

In [131]:
mean_country_per_auction = num_countries_per_auction.groupby("bidder_id").mean()
mean_country_per_auction = mean_country_per_auction.reset_index().rename(columns = {'country':"mean_country_per_auction"})

max_country_per_auction = num_countries_per_auction.groupby("bidder_id").max()
max_country_per_auction = max_country_per_auction.reset_index().rename(columns = {'country':"max_country_per_auction"})

min_country_per_auction = num_countries_per_auction.groupby("bidder_id").min()
min_country_per_auction = min_country_per_auction.reset_index().rename(columns = {'country':"min_country_per_auction"})

std_country_per_auction = num_countries_per_auction.groupby("bidder_id").std()
std_country_per_auction = std_country_per_auction.reset_index().rename(columns = {'country':"std_country_per_auction"})


In [132]:
country_bids = pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [133]:
country_bids = country_bids.merge(mean_country_per_auction,on='bidder_id',how='left')
country_bids = country_bids.merge(max_country_per_auction,on='bidder_id',how='left')
country_bids = country_bids.merge(min_country_per_auction,on='bidder_id',how='left')
country_bids = country_bids.merge(std_country_per_auction,on='bidder_id',how='left')

In [134]:
device_bids = bids_df.sort_values(['bidder_id','device'])
num_devices_per_auction = device_bids.groupby(['bidder_id','auction'])['device'].nunique()
num_devices_per_auction = pd.DataFrame(num_devices_per_auction)

In [135]:
mean_devices_per_auction = num_devices_per_auction.groupby("bidder_id").mean()
mean_devices_per_auction = mean_devices_per_auction.reset_index().rename(columns = {'device':"mean_devices_per_auction"})

max_devices_per_auction = num_devices_per_auction.groupby("bidder_id").max()
max_devices_per_auction = max_devices_per_auction.reset_index().rename(columns = {'device':"max_devices_per_auction"})

min_devices_per_auction = num_devices_per_auction.groupby("bidder_id").min()
min_devices_per_auction = min_devices_per_auction.reset_index().rename(columns = {'device':"min_devices_per_auction"})

std_devices_per_auction = num_devices_per_auction.groupby("bidder_id").std()
std_devices_per_auction = std_devices_per_auction.reset_index().rename(columns = {'device':"std_devices_per_auction"})

In [136]:
device_bids = pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [137]:
device_bids = device_bids.merge(mean_devices_per_auction,on='bidder_id',how='left')
device_bids = device_bids.merge(max_devices_per_auction,on='bidder_id',how='left')
device_bids = device_bids.merge(min_devices_per_auction,on='bidder_id',how='left')
device_bids = device_bids.merge(std_devices_per_auction,on='bidder_id',how='left')

In [138]:
device_bids.fillna(0,inplace=True)

In [139]:
ip_bids = bids_df.sort_values(['bidder_id','ip'])
num_ip_per_auction = ip_bids.groupby(['bidder_id','auction'])['ip'].nunique()
num_ip_per_auction = pd.DataFrame(num_ip_per_auction)

In [140]:
mean_ip_per_auction = num_ip_per_auction.groupby("bidder_id").mean()
mean_ip_per_auction = mean_ip_per_auction.reset_index().rename(columns = {'ip':"mean_ip_per_auction"})

max_ip_per_auction = num_ip_per_auction.groupby("bidder_id").max()
max_ip_per_auction = max_ip_per_auction.reset_index().rename(columns = {'ip':"max_ip_per_auction"})

min_ip_per_auction = num_ip_per_auction.groupby("bidder_id").min()
min_ip_per_auction = min_ip_per_auction.reset_index().rename(columns = {'ip':"min_ip_per_auction"})

std_ip_per_auction = num_ip_per_auction.groupby("bidder_id").std()
std_ip_per_auction = std_ip_per_auction.reset_index().rename(columns = {'ip':"std_ip_per_auction"})

In [141]:
ip_bids = pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [142]:
ip_bids = ip_bids.merge(mean_ip_per_auction,on='bidder_id',how='left')
ip_bids = ip_bids.merge(max_ip_per_auction,on='bidder_id',how='left')
ip_bids = ip_bids.merge(min_ip_per_auction,on='bidder_id',how='left')
ip_bids = ip_bids.merge(std_ip_per_auction,on='bidder_id',how='left')

In [143]:
ip_bids.fillna(0,inplace=True)

In [144]:
url_bids= bids_df.sort_values(['bidder_id','url'])
num_url_per_auction = url_bids.groupby(['bidder_id','auction'])['url'].nunique()
num_url_per_auction = pd.DataFrame(num_url_per_auction)

In [145]:
mean_url_per_auction = num_url_per_auction.groupby("bidder_id").mean()
mean_url_per_auction = mean_url_per_auction.reset_index().rename(columns = {'url':"mean_url_per_auction"})

max_url_per_auction = num_url_per_auction.groupby("bidder_id").max()
max_url_per_auction = max_url_per_auction.reset_index().rename(columns = {'url':"max_url_per_auction"})

min_url_per_auction = num_url_per_auction.groupby("bidder_id").min()
min_url_per_auction = min_url_per_auction.reset_index().rename(columns = {'url':"min_url_per_auction"})

std_url_per_auction = num_url_per_auction.groupby("bidder_id").std()
std_url_per_auction = std_url_per_auction.reset_index().rename(columns = {'url':"std_url_per_auction"})


In [146]:
url_bids = pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [147]:
url_bids = url_bids.merge(mean_url_per_auction,on='bidder_id',how='left')
url_bids = url_bids.merge(max_url_per_auction,on='bidder_id',how='left')
url_bids = url_bids.merge(min_url_per_auction,on='bidder_id',how='left')
url_bids = url_bids.merge(std_url_per_auction,on='bidder_id',how='left')

In [148]:
url_bids.fillna(0,inplace=True)

# Making of training dataset

In [149]:
train_model= new_train.copy()

In [150]:
train_model= train_model.merge(country_bids,on='bidder_id',how='left')

In [151]:
train_model= train_model.merge(device_bids,on='bidder_id',how='left')

In [152]:
train_model= train_model.merge(ip_bids,on='bidder_id',how='left')

In [153]:
train_model= train_model.merge(url_bids,on='bidder_id',how='left')

In [154]:
try_auction = bids_df.groupby(['auction'])

# Checking of how many 'first's and 'last's bids each bidder made

In [155]:
first_bid_list = set(try_auction.first()['bidder_id'])
last_bid_list = set(try_auction.last()['bidder_id'])

In [156]:
def check_first_bid(dummy):
    count = 0
    for bidder_id in first_bid_list:
        if(dummy == bidder_id):
            count+=1
    return count
        
def check_last_bid(dummy):
    count = 0
    for bidder_id in last_bid_list:
        if(dummy == bidder_id):
            count+=1
    return count
        
    

In [157]:
train_model['check_first'] = train_model['bidder_id'].apply(func = check_first_bid)
train_model['check_last'] = train_model['bidder_id'].apply(func = check_last_bid)

In [158]:
train_model.fillna(0,inplace=True)

In [159]:
bots_train_model = train_model[train_model['outcome'] == 1]
humans_train_model = train_model[train_model['outcome'] == 0]
bots_train_model = bots_train_model[bots_train_model['num_bids'] > 1]

In [160]:
train_model = pd.concat([bots_train_model,humans_train_model])

In [161]:
train_model = train_model.reset_index()

In [162]:
train_model.drop(columns = ['index'],inplace = True)

# Training of model with training data set and testing out features

In [164]:
y = train_model['outcome']
X = train_model.drop(columns= [
                                     'outcome', 
                                     'bidder_id',
                                     'payment_account',
                                     'address', 
                                     
                                    
                                     ])

In [165]:
model1 = RandomForestClassifier(n_estimators=500, max_features=35,
                                   max_depth=8, random_state=20,
                                   criterion='entropy',)

model2 = xgb.XGBClassifier( objective= 'binary:logistic', nthread= 10,
                              eval_metric= 'auc', silent= 1, seed= 20,

                              max_depth= 6, gamma= 0, base_score= 0.50,
                              min_child_weight= 4, subsample= 0.5,
                              colsample_bytree= 1, eta= 0.01,)

model3 = GradientBoostingClassifier(n_estimators=200,
                                       random_state=20,
                                       max_depth=5,
                                       learning_rate=0.03,
                                       max_features=5,)
model4 = CatBoostClassifier(random_state=20)

k_fold_accuracy = []
kf = KFold(n_splits=10,shuffle = True)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model1.fit(X_train,y_train)
    model2.fit(X_train,y_train)
    model3.fit(X_train,y_train)
    model4.fit(X_train,y_train)


    y_pred1 = model1.predict_proba(X_test)
    y_pred2 = model2.predict_proba(X_test)
    y_pred3 = model3.predict_proba(X_test)
    y_pred4 = model4.predict_proba(X_test)

    
    accu1 = model1.predict(X_test)
    accu2 = model2.predict(X_test)
    accu3 = model3.predict(X_test)
    accu4 = model4.predict(X_test)

C:\Users\user\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:44:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Learning rate set to 0.013263
0:	learn: 0.6711072	total: 41.8ms	remaining: 41.8s
1:	learn: 0.6509545	total: 48.3ms	remaining: 24.1s
2:	learn: 0.6278112	total: 54.7ms	remaining: 18.2s
3:	learn: 0.6016171	total: 61.5ms	remaining: 15.3s
4:	learn: 0.5832346	total: 67.8ms	remaining: 13.5s
5:	learn: 0.5604257	total: 74ms	remaining: 12.3s
6:	learn: 0.5378046	total: 80.3ms	remaining: 11.4s
7:	learn: 0.5205258	total: 84.4ms	remaining: 10.5s
8:	learn: 0.4994688	total: 90.9ms	remaining: 10s
9:	learn: 0.4836131	total: 97.4ms	remaining: 9.64s
10:	learn: 0.4650155	total: 104ms	remaining: 9.35s
11:	learn: 0.4476382	total: 1

171:	learn: 0.0796829	total: 1.21s	remaining: 5.84s
172:	learn: 0.0792129	total: 1.22s	remaining: 5.83s
173:	learn: 0.0790246	total: 1.23s	remaining: 5.83s
174:	learn: 0.0787003	total: 1.24s	remaining: 5.82s
175:	learn: 0.0783411	total: 1.24s	remaining: 5.81s
176:	learn: 0.0781382	total: 1.25s	remaining: 5.8s
177:	learn: 0.0778063	total: 1.25s	remaining: 5.79s
178:	learn: 0.0775571	total: 1.26s	remaining: 5.78s
179:	learn: 0.0772997	total: 1.27s	remaining: 5.77s
180:	learn: 0.0770182	total: 1.27s	remaining: 5.76s
181:	learn: 0.0767715	total: 1.28s	remaining: 5.75s
182:	learn: 0.0765886	total: 1.29s	remaining: 5.74s
183:	learn: 0.0763383	total: 1.29s	remaining: 5.73s
184:	learn: 0.0761468	total: 1.3s	remaining: 5.72s
185:	learn: 0.0757742	total: 1.3s	remaining: 5.71s
186:	learn: 0.0753957	total: 1.31s	remaining: 5.71s
187:	learn: 0.0751529	total: 1.32s	remaining: 5.7s
188:	learn: 0.0749873	total: 1.32s	remaining: 5.68s
189:	learn: 0.0747673	total: 1.33s	remaining: 5.67s
190:	learn: 0.07

333:	learn: 0.0507428	total: 2.28s	remaining: 4.55s
334:	learn: 0.0506619	total: 2.29s	remaining: 4.54s
335:	learn: 0.0505032	total: 2.29s	remaining: 4.53s
336:	learn: 0.0503694	total: 2.3s	remaining: 4.53s
337:	learn: 0.0502441	total: 2.31s	remaining: 4.52s
338:	learn: 0.0501237	total: 2.31s	remaining: 4.51s
339:	learn: 0.0499444	total: 2.32s	remaining: 4.51s
340:	learn: 0.0497594	total: 2.33s	remaining: 4.5s
341:	learn: 0.0496745	total: 2.33s	remaining: 4.49s
342:	learn: 0.0495909	total: 2.34s	remaining: 4.49s
343:	learn: 0.0494983	total: 2.35s	remaining: 4.48s
344:	learn: 0.0494351	total: 2.35s	remaining: 4.47s
345:	learn: 0.0493204	total: 2.36s	remaining: 4.46s
346:	learn: 0.0491802	total: 2.37s	remaining: 4.46s
347:	learn: 0.0491045	total: 2.37s	remaining: 4.45s
348:	learn: 0.0489461	total: 2.38s	remaining: 4.44s
349:	learn: 0.0488050	total: 2.39s	remaining: 4.43s
350:	learn: 0.0487749	total: 2.39s	remaining: 4.42s
351:	learn: 0.0486005	total: 2.4s	remaining: 4.42s
352:	learn: 0.0

493:	learn: 0.0364987	total: 3.35s	remaining: 3.43s
494:	learn: 0.0363831	total: 3.35s	remaining: 3.42s
495:	learn: 0.0363385	total: 3.36s	remaining: 3.42s
496:	learn: 0.0363085	total: 3.37s	remaining: 3.41s
497:	learn: 0.0362832	total: 3.38s	remaining: 3.4s
498:	learn: 0.0361973	total: 3.38s	remaining: 3.4s
499:	learn: 0.0361212	total: 3.39s	remaining: 3.39s
500:	learn: 0.0360198	total: 3.4s	remaining: 3.38s
501:	learn: 0.0359545	total: 3.4s	remaining: 3.38s
502:	learn: 0.0358860	total: 3.41s	remaining: 3.37s
503:	learn: 0.0358301	total: 3.42s	remaining: 3.36s
504:	learn: 0.0358014	total: 3.42s	remaining: 3.35s
505:	learn: 0.0357383	total: 3.43s	remaining: 3.35s
506:	learn: 0.0356987	total: 3.44s	remaining: 3.34s
507:	learn: 0.0356792	total: 3.44s	remaining: 3.33s
508:	learn: 0.0356397	total: 3.45s	remaining: 3.33s
509:	learn: 0.0355806	total: 3.45s	remaining: 3.32s
510:	learn: 0.0354804	total: 3.46s	remaining: 3.31s
511:	learn: 0.0354201	total: 3.47s	remaining: 3.3s
512:	learn: 0.035

...

164:	learn: 0.0757870	total: 1.25s	remaining: 6.34s
165:	learn: 0.0755521	total: 1.26s	remaining: 6.33s
166:	learn: 0.0752243	total: 1.27s	remaining: 6.32s
167:	learn: 0.0748935	total: 1.27s	remaining: 6.31s
168:	learn: 0.0747001	total: 1.28s	remaining: 6.3s
169:	learn: 0.0744832	total: 1.29s	remaining: 6.29s
170:	learn: 0.0741740	total: 1.3s	remaining: 6.29s
171:	learn: 0.0738771	total: 1.3s	remaining: 6.28s
172:	learn: 0.0736223	total: 1.31s	remaining: 6.27s
173:	learn: 0.0734703	total: 1.32s	remaining: 6.26s
174:	learn: 0.0732091	total: 1.33s	remaining: 6.26s
175:	learn: 0.0728926	total: 1.33s	remaining: 6.25s
176:	learn: 0.0725952	total: 1.34s	remaining: 6.24s
177:	learn: 0.0722965	total: 1.35s	remaining: 6.24s
178:	learn: 0.0719357	total: 1.36s	remaining: 6.23s
179:	learn: 0.0717391	total: 1.36s	remaining: 6.22s
180:	learn: 0.0715445	total: 1.37s	remaining: 6.21s
181:	learn: 0.0713390	total: 1.38s	remaining: 6.21s
182:	learn: 0.0710135	total: 1.39s	remaining: 6.2s
183:	learn: 0.07

325:	learn: 0.0480280	total: 2.5s	remaining: 5.17s
326:	learn: 0.0479696	total: 2.51s	remaining: 5.16s
327:	learn: 0.0478456	total: 2.52s	remaining: 5.16s
328:	learn: 0.0477945	total: 2.52s	remaining: 5.15s
329:	learn: 0.0477121	total: 2.53s	remaining: 5.14s
330:	learn: 0.0474905	total: 2.54s	remaining: 5.14s
331:	learn: 0.0474273	total: 2.55s	remaining: 5.13s
332:	learn: 0.0472333	total: 2.56s	remaining: 5.12s
333:	learn: 0.0471858	total: 2.57s	remaining: 5.12s
334:	learn: 0.0470515	total: 2.57s	remaining: 5.11s
335:	learn: 0.0468932	total: 2.58s	remaining: 5.1s
336:	learn: 0.0467769	total: 2.59s	remaining: 5.1s
337:	learn: 0.0466677	total: 2.6s	remaining: 5.09s
338:	learn: 0.0465510	total: 2.61s	remaining: 5.08s
339:	learn: 0.0464370	total: 2.62s	remaining: 5.08s
340:	learn: 0.0462562	total: 2.62s	remaining: 5.07s
341:	learn: 0.0460840	total: 2.63s	remaining: 5.06s
342:	learn: 0.0460324	total: 2.64s	remaining: 5.06s
343:	learn: 0.0459472	total: 2.65s	remaining: 5.05s
344:	learn: 0.04

506:	learn: 0.0334633	total: 3.92s	remaining: 3.82s
507:	learn: 0.0334467	total: 3.93s	remaining: 3.81s
508:	learn: 0.0333667	total: 3.94s	remaining: 3.8s
509:	learn: 0.0332750	total: 3.95s	remaining: 3.79s
510:	learn: 0.0332451	total: 3.95s	remaining: 3.78s
511:	learn: 0.0331645	total: 3.96s	remaining: 3.78s
512:	learn: 0.0330859	total: 3.97s	remaining: 3.77s
513:	learn: 0.0330099	total: 3.98s	remaining: 3.76s
514:	learn: 0.0329802	total: 3.98s	remaining: 3.75s
515:	learn: 0.0329068	total: 3.99s	remaining: 3.74s
516:	learn: 0.0328886	total: 4s	remaining: 3.74s
517:	learn: 0.0328071	total: 4.01s	remaining: 3.73s
518:	learn: 0.0327404	total: 4.01s	remaining: 3.72s
519:	learn: 0.0326407	total: 4.02s	remaining: 3.71s
520:	learn: 0.0325606	total: 4.03s	remaining: 3.71s
521:	learn: 0.0324751	total: 4.04s	remaining: 3.7s
522:	learn: 0.0324049	total: 4.05s	remaining: 3.69s
523:	learn: 0.0323482	total: 4.05s	remaining: 3.68s
524:	learn: 0.0323282	total: 4.06s	remaining: 3.67s
525:	learn: 0.032

667:	learn: 0.0257029	total: 5.17s	remaining: 2.57s
668:	learn: 0.0256539	total: 5.17s	remaining: 2.56s
669:	learn: 0.0256254	total: 5.18s	remaining: 2.55s
670:	learn: 0.0255841	total: 5.19s	remaining: 2.54s
671:	learn: 0.0255759	total: 5.2s	remaining: 2.54s
672:	learn: 0.0255118	total: 5.2s	remaining: 2.53s
673:	learn: 0.0254633	total: 5.21s	remaining: 2.52s
674:	learn: 0.0254318	total: 5.22s	remaining: 2.51s
675:	learn: 0.0254227	total: 5.23s	remaining: 2.5s
676:	learn: 0.0253980	total: 5.23s	remaining: 2.5s
677:	learn: 0.0253456	total: 5.24s	remaining: 2.49s
678:	learn: 0.0253359	total: 5.25s	remaining: 2.48s
679:	learn: 0.0252860	total: 5.26s	remaining: 2.47s
680:	learn: 0.0252661	total: 5.26s	remaining: 2.46s
681:	learn: 0.0251966	total: 5.27s	remaining: 2.46s
682:	learn: 0.0251741	total: 5.28s	remaining: 2.45s
683:	learn: 0.0251421	total: 5.29s	remaining: 2.44s
684:	learn: 0.0251271	total: 5.29s	remaining: 2.43s
685:	learn: 0.0250958	total: 5.3s	remaining: 2.43s
686:	learn: 0.025

841:	learn: 0.0199732	total: 6.59s	remaining: 1.24s
842:	learn: 0.0199416	total: 6.6s	remaining: 1.23s
843:	learn: 0.0198964	total: 6.61s	remaining: 1.22s
844:	learn: 0.0198522	total: 6.61s	remaining: 1.21s
845:	learn: 0.0198169	total: 6.62s	remaining: 1.21s
846:	learn: 0.0197841	total: 6.63s	remaining: 1.2s
847:	learn: 0.0197499	total: 6.64s	remaining: 1.19s
848:	learn: 0.0196847	total: 6.64s	remaining: 1.18s
849:	learn: 0.0196559	total: 6.65s	remaining: 1.17s
850:	learn: 0.0195956	total: 6.66s	remaining: 1.17s
851:	learn: 0.0195652	total: 6.67s	remaining: 1.16s
852:	learn: 0.0195397	total: 6.67s	remaining: 1.15s
853:	learn: 0.0195271	total: 6.68s	remaining: 1.14s
854:	learn: 0.0195197	total: 6.69s	remaining: 1.13s
855:	learn: 0.0195155	total: 6.7s	remaining: 1.13s
856:	learn: 0.0194793	total: 6.71s	remaining: 1.12s
857:	learn: 0.0194546	total: 6.71s	remaining: 1.11s
858:	learn: 0.0194133	total: 6.72s	remaining: 1.1s
859:	learn: 0.0194084	total: 6.73s	remaining: 1.09s
860:	learn: 0.01

# Analysing of Model Results

In [166]:
print('\nAccuracy_score on RandomForest : ', accuracy_score(y_test, accu1))
print('\nAccuracy_score on XGB : ', accuracy_score(y_test, accu2))
print('\nAccuracy_score on GradientBoost : ', accuracy_score(y_test, accu3))
print('\nAccuracy_score on CatBoost : ', accuracy_score(y_test, accu4))

averaging_accuracy = (accuracy_score(y_test, accu1) + accuracy_score(y_test, accu2) +accuracy_score(y_test, accu3)+accuracy_score(y_test, accu4))/4
print('\nAverage Accuracy : ', averaging_accuracy)



Accuracy_score on RandomForest :  0.945

Accuracy_score on XGB :  0.935

Accuracy_score on GradientBoost :  0.95

Accuracy_score on CatBoost :  0.935

Average Accuracy :  0.94125


In [167]:
print("AUC SCORES")
print("-----------------------------------")
print("RandomForest : " , roc_auc_score(y_test,y_pred1[:,1]))

print("XGB : " , roc_auc_score(y_test,y_pred2[:,1]))
print("Gradient Boost : " , roc_auc_score(y_test,y_pred3[:,1]))
print("Cat Boost : ", roc_auc_score(y_test,y_pred4[:,1]))

AUC SCORES
-----------------------------------
RandomForest :  0.9461279461279462
XGB :  0.9278499278499279
Gradient Boost :  0.9422799422799423
Cat Boost :  0.9413179413179413


In [168]:
score1 = roc_auc_score(y_test,y_pred1[:,1])
score2 = roc_auc_score(y_test,y_pred2[:,1])
score3 = roc_auc_score(y_test,y_pred3[:,1])
score4 = roc_auc_score(y_test,y_pred4[:,1])

In [169]:
print("\nAverage AUC :" , (score1+score2+score3+score4)/4)


Average AUC : 0.9393939393939394


# Executing predictions on final test dataset

In [170]:
final_test_df = test_df.merge(bids_unique,how= 'left',on = 'bidder_id')

In [171]:
final_test_df = final_test_df.merge(counts, how = 'left', on='bidder_id')

In [172]:
final_test_df = final_test_df.merge(bids,how='left', on ='bidder_id')

In [173]:
final_test_df.fillna(0,inplace=True)

In [174]:
final_test_df['country'].replace({0:1},inplace=True)

In [175]:
final_test_df['bids_per_auction'] = final_test_df['num_bids']/final_test_df['auction']
final_test_df['bids_per_url'] = final_test_df['num_bids']/final_test_df['url']
final_test_df['bids_per_ip'] = final_test_df['num_bids']/final_test_df['ip']
final_test_df['bids_per_country'] = final_test_df['num_bids']/final_test_df['country']
final_test_df['bids_per_device'] = final_test_df['num_bids']/final_test_df['device']

In [176]:
final_test_df = final_test_df.merge(country_bids,on='bidder_id',how='left')
final_test_df = final_test_df.merge(device_bids,on='bidder_id',how='left')
final_test_df = final_test_df.merge(ip_bids,on='bidder_id',how='left')
final_test_df = final_test_df.merge(url_bids,on='bidder_id',how='left')

In [177]:
final_test_df['check_first'] = final_test_df['bidder_id'].apply(func = check_first_bid)
final_test_df['check_last'] = final_test_df['bidder_id'].apply(func = check_last_bid)

In [178]:
testing_feature =['auction',
       'merchandise', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'maxtime_num', 'mintime_num', 'maxdiff_num', 'mindiff_num',
       'meandiff_num', 'mediandiff_num', 'std_diff', 'median_start',
       'mean_start', 'std_start', 'median_end', 'mean_end', 'std_end',
       'mean_entropy', 'median_entropy', 'std_entropy',
       'median_ips_per_bidder_per_auction', 'mean_ips_per_bidder_per_auction',
       'std_ips_per_bidder_per_auction', 'ip_entropy',
       'median_country_per_bidder_per_auction',
       'mean_country_per_bidder_per_auction',
       'std_country_per_bidder_per_auction', 'country_entropy',
       'median_devices_per_bidder_per_auction',
       'mean_devices_per_bidder_per_auction',
       'std_devices_per_bidder_per_auction', 'device_entropy',
       'median_url_per_bidder_per_auction', 'mean_url_per_bidder_per_auction',
       'std_url_per_bidder_per_auction', 'url_entropy', 'bids_per_auction',
       'bids_per_url', 'bids_per_ip', 'bids_per_country', 'bids_per_device',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction', 'check_first',
       'check_last']

In [179]:
X_train = X
y_train = y

In [180]:
X_test = final_test_df[testing_feature]
X_test.fillna(0, inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [181]:
fmodel1= RandomForestClassifier(n_estimators=500, max_features=35,
                                   max_depth=8, random_state=20,
                                   criterion='entropy',)
fmodel2= GradientBoostingClassifier(n_estimators=200,
                                       random_state=20,
                                       max_depth=5,
                                       learning_rate=0.03,
                                       max_features=5,)
fmodel3 = xgb.XGBClassifier( objective= 'binary:logistic', nthread= 10,
                              eval_metric= 'auc', silent= 1, seed= 20,

                              max_depth= 6, gamma= 0, base_score= 0.50,
                              min_child_weight= 4, subsample= 0.5,
                              colsample_bytree= 1, eta= 0.01,)
fmodel4= CatBoostClassifier(random_state=20)


fmodel1=fmodel1.fit(X_train, y_train)
fmodel2=fmodel2.fit(X_train,y_train)
fmodel3=fmodel3.fit(X_train,y_train)
fmodel4=fmodel4.fit(X_train, y_train)

final_answer1= fmodel1.predict_proba(X_test)[:,1]
final_answer2= fmodel2.predict_proba(X_test)[:,1]
final_answer3= fmodel3.predict_proba(X_test)[:,1]
final_answer4= fmodel4.predict_proba(X_test)[:,1]


finalpredd= (final_answer1+final_answer2+final_answer3+final_answer4)/4
print(len(finalpredd))

C:\Users\user\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:47:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Learning rate set to 0.013874
0:	learn: 0.6696905	total: 24.4ms	remaining: 24.3s
1:	learn: 0.6485685	total: 32.8ms	remaining: 16.4s
2:	learn: 0.6238663	total: 40.7ms	remaining: 13.5s
3:	learn: 0.5964865	total: 53.7ms	remaining: 13.4s
4:	learn: 0.5770670	total: 66.8ms	remaining: 13.3s
5:	learn: 0.5521168	total: 76.6ms	remaining: 12.7s
6:	learn: 0.5281566	total: 85.6ms	remaining: 12.1s
7:	learn: 0.5098498	total: 92ms	remaining: 11.4s
8:	learn: 0.4881038	total: 101ms	remaining: 11.1s
9:	learn: 0.4721672	total: 109ms	remaining: 10.7s
10:	learn: 0.4531617	total: 117ms	remaining: 10.5s
11:	learn: 0.4356077	total: 1

160:	learn: 0.0775513	total: 1.25s	remaining: 6.51s
161:	learn: 0.0773347	total: 1.26s	remaining: 6.5s
162:	learn: 0.0770650	total: 1.26s	remaining: 6.49s
163:	learn: 0.0768746	total: 1.27s	remaining: 6.49s
164:	learn: 0.0765519	total: 1.28s	remaining: 6.48s
165:	learn: 0.0762661	total: 1.29s	remaining: 6.47s
166:	learn: 0.0759467	total: 1.3s	remaining: 6.46s
167:	learn: 0.0755709	total: 1.3s	remaining: 6.46s
168:	learn: 0.0753897	total: 1.31s	remaining: 6.45s
169:	learn: 0.0751457	total: 1.32s	remaining: 6.44s
170:	learn: 0.0749674	total: 1.33s	remaining: 6.44s
171:	learn: 0.0745885	total: 1.34s	remaining: 6.44s
172:	learn: 0.0743795	total: 1.34s	remaining: 6.43s
173:	learn: 0.0741299	total: 1.35s	remaining: 6.43s
174:	learn: 0.0738979	total: 1.36s	remaining: 6.42s
175:	learn: 0.0736171	total: 1.37s	remaining: 6.42s
176:	learn: 0.0734214	total: 1.38s	remaining: 6.41s
177:	learn: 0.0730612	total: 1.39s	remaining: 6.4s
178:	learn: 0.0727932	total: 1.39s	remaining: 6.39s
179:	learn: 0.07

340:	learn: 0.0476616	total: 2.67s	remaining: 5.17s
341:	learn: 0.0475931	total: 2.68s	remaining: 5.16s
342:	learn: 0.0475197	total: 2.69s	remaining: 5.15s
343:	learn: 0.0473461	total: 2.7s	remaining: 5.14s
344:	learn: 0.0472482	total: 2.71s	remaining: 5.14s
345:	learn: 0.0470932	total: 2.71s	remaining: 5.13s
346:	learn: 0.0469500	total: 2.72s	remaining: 5.12s
347:	learn: 0.0468544	total: 2.73s	remaining: 5.11s
348:	learn: 0.0468016	total: 2.74s	remaining: 5.1s
349:	learn: 0.0467201	total: 2.74s	remaining: 5.1s
350:	learn: 0.0465299	total: 2.75s	remaining: 5.09s
351:	learn: 0.0464924	total: 2.76s	remaining: 5.08s
352:	learn: 0.0463814	total: 2.77s	remaining: 5.07s
353:	learn: 0.0462484	total: 2.77s	remaining: 5.07s
354:	learn: 0.0461583	total: 2.78s	remaining: 5.06s
355:	learn: 0.0461112	total: 2.79s	remaining: 5.05s
356:	learn: 0.0459325	total: 2.8s	remaining: 5.04s
357:	learn: 0.0457772	total: 2.81s	remaining: 5.04s
358:	learn: 0.0456862	total: 2.81s	remaining: 5.03s
359:	learn: 0.04

521:	learn: 0.0333868	total: 4.1s	remaining: 3.76s
522:	learn: 0.0333603	total: 4.11s	remaining: 3.75s
523:	learn: 0.0333098	total: 4.12s	remaining: 3.74s
524:	learn: 0.0332921	total: 4.13s	remaining: 3.73s
525:	learn: 0.0332112	total: 4.13s	remaining: 3.73s
526:	learn: 0.0331655	total: 4.14s	remaining: 3.72s
527:	learn: 0.0331342	total: 4.15s	remaining: 3.71s
528:	learn: 0.0330501	total: 4.16s	remaining: 3.7s
529:	learn: 0.0329906	total: 4.16s	remaining: 3.69s
530:	learn: 0.0329263	total: 4.17s	remaining: 3.69s
531:	learn: 0.0328637	total: 4.18s	remaining: 3.68s
532:	learn: 0.0328209	total: 4.19s	remaining: 3.67s
533:	learn: 0.0327417	total: 4.2s	remaining: 3.66s
534:	learn: 0.0326944	total: 4.2s	remaining: 3.65s
535:	learn: 0.0326778	total: 4.21s	remaining: 3.65s
536:	learn: 0.0326621	total: 4.22s	remaining: 3.64s
537:	learn: 0.0326358	total: 4.23s	remaining: 3.63s
538:	learn: 0.0325595	total: 4.24s	remaining: 3.62s
539:	learn: 0.0325207	total: 4.24s	remaining: 3.62s
540:	learn: 0.03

701:	learn: 0.0252591	total: 5.53s	remaining: 2.35s
702:	learn: 0.0252108	total: 5.53s	remaining: 2.34s
703:	learn: 0.0251503	total: 5.54s	remaining: 2.33s
704:	learn: 0.0251208	total: 5.55s	remaining: 2.32s
705:	learn: 0.0250694	total: 5.56s	remaining: 2.31s
706:	learn: 0.0250160	total: 5.56s	remaining: 2.31s
707:	learn: 0.0249612	total: 5.57s	remaining: 2.3s
708:	learn: 0.0249357	total: 5.58s	remaining: 2.29s
709:	learn: 0.0248785	total: 5.59s	remaining: 2.28s
710:	learn: 0.0248241	total: 5.59s	remaining: 2.27s
711:	learn: 0.0247905	total: 5.6s	remaining: 2.27s
712:	learn: 0.0247493	total: 5.61s	remaining: 2.26s
713:	learn: 0.0246958	total: 5.62s	remaining: 2.25s
714:	learn: 0.0246501	total: 5.63s	remaining: 2.24s
715:	learn: 0.0245906	total: 5.63s	remaining: 2.23s
716:	learn: 0.0245336	total: 5.64s	remaining: 2.23s
717:	learn: 0.0244710	total: 5.65s	remaining: 2.22s
718:	learn: 0.0244249	total: 5.66s	remaining: 2.21s
719:	learn: 0.0244037	total: 5.66s	remaining: 2.2s
720:	learn: 0.0

882:	learn: 0.0177032	total: 6.95s	remaining: 921ms
883:	learn: 0.0176734	total: 6.96s	remaining: 913ms
884:	learn: 0.0176564	total: 6.97s	remaining: 905ms
885:	learn: 0.0176339	total: 6.97s	remaining: 897ms
886:	learn: 0.0175979	total: 6.98s	remaining: 890ms
887:	learn: 0.0175730	total: 6.99s	remaining: 882ms
888:	learn: 0.0175502	total: 7s	remaining: 874ms
889:	learn: 0.0175135	total: 7s	remaining: 866ms
890:	learn: 0.0174977	total: 7.01s	remaining: 858ms
891:	learn: 0.0174573	total: 7.02s	remaining: 850ms
892:	learn: 0.0174127	total: 7.03s	remaining: 842ms
893:	learn: 0.0173652	total: 7.04s	remaining: 834ms
894:	learn: 0.0173247	total: 7.04s	remaining: 827ms
895:	learn: 0.0172984	total: 7.05s	remaining: 819ms
896:	learn: 0.0172820	total: 7.06s	remaining: 811ms
897:	learn: 0.0172474	total: 7.07s	remaining: 803ms
898:	learn: 0.0172319	total: 7.08s	remaining: 795ms
899:	learn: 0.0172126	total: 7.08s	remaining: 787ms
900:	learn: 0.0171911	total: 7.09s	remaining: 779ms
901:	learn: 0.0171

In [182]:
output_df= pd.DataFrame({
    'bidder_id': final_test_df.bidder_id,
    'Prediction':finalpredd
})
output_df

,bidder_id,Prediction
0,49bb5a3c944b8fc337981cc7a9ccae41u31d7,0.046944
1,a921612b85a1494456e74c09393ccb65ylp4y,0.047130
2,6b601e72a4d264dab9ace9d7b229b47479v6i,0.058043
3,eaf0ed0afc9689779417274b4791726cn5udi,0.047536
4,cdecd8d02ed8c6037e38042c7745f688mx5sf,0.054016
5,d4aed439bdc854a56fc6cc3bdb986775w7hxw,0.068339
6,ed591299b162a19ff77f0479495831b31hl1q,0.047042
7,eebdee08b0f67283126ef60307f49680sb9va,0.061655
8,6887f0abc4eb4c79eb0e23c48ceea186vjfih,0.047082
9,37eb6e2979e66d4ce29a74ac1c8bc6a5lqs6t,0.057871


In [183]:
output_df.to_csv('facebooksubmission.csv', index= False)